# 06. Results Analysis

Analyze and visualize experiment results from MNIST and CUB-200 experiments.

In [ ]:
import sys, os, json, glob
if os.path.basename(os.getcwd()) == 'notebooks': os.chdir('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

## Load Results

In [ ]:
# Find latest results
mnist_files = sorted(glob.glob('./experiments/mnist_results_*/mnist_results.json'))
cub_files = sorted(glob.glob('./experiments/cub200_results_*/cub200_results.json'))

mnist_results = []
cub_results = []

if mnist_files:
    with open(mnist_files[-1]) as f:
        mnist_results = json.load(f)
    print(f"Loaded MNIST: {mnist_files[-1]} ({len(mnist_results)} configs)")

if cub_files:
    with open(cub_files[-1]) as f:
        cub_results = json.load(f)
    print(f"Loaded CUB-200: {cub_files[-1]} ({len(cub_results)} configs)")

## MNIST Analysis

In [ ]:
if mnist_results:
    df = pd.DataFrame(mnist_results)
    df['accuracy_pct'] = df['mean_accuracy'] * 100
    
    # Pivot table: Attack vs Defense
    for dist in df['distribution'].unique():
        subset = df[df['distribution'] == dist]
        pivot = subset.pivot_table(values='accuracy_pct', index='attack', columns='defense', aggfunc='mean')
        
        plt.figure(figsize=(10, 6))
        sns.heatmap(pivot, annot=True, fmt='.1f', cmap='RdYlGn', vmin=80, vmax=100)
        alpha = subset['alpha'].iloc[0]
        title = 'IID' if dist == 'iid' else f'Non-IID (α={alpha})'
        plt.title(f'MNIST Accuracy: {title}')
        plt.tight_layout()
        plt.show()

## CUB-200 Analysis

In [ ]:
if cub_results:
    df = pd.DataFrame(cub_results)
    acc_key = 'mean_accuracy' if 'mean_accuracy' in df.columns else 'mean_acc'
    df['accuracy_pct'] = df[acc_key] * 100
    
    print("\nCUB-200 Results Summary:")
    print(df[['attack', 'defense', 'accuracy_pct']].to_string())

## Comparison Chart

In [ ]:
if mnist_results:
    df = pd.DataFrame(mnist_results)
    iid = df[df['distribution'] == 'iid'].copy()
    
    fig, ax = plt.subplots(figsize=(12, 6))
    x = range(len(iid))
    bars = ax.bar(x, iid['mean_accuracy']*100, yerr=iid['std_accuracy']*100, capsize=3)
    ax.set_xticks(x)
    ax.set_xticklabels([f"{r['attack']}\n+{r['defense']}" for _, r in iid.iterrows()], rotation=45, ha='right')
    ax.set_ylabel('Accuracy (%)')
    ax.set_title('MNIST IID: Attack vs Defense Combinations')
    ax.set_ylim(0, 100)
    plt.tight_layout()
    plt.show()

## Export Summary

In [ ]:
# Combine and export
all_data = []
for r in mnist_results:
    r['dataset'] = 'MNIST'
    all_data.append(r)
for r in cub_results:
    r['dataset'] = 'CUB-200'
    all_data.append(r)

if all_data:
    df = pd.DataFrame(all_data)
    df.to_csv('./experiments/combined_results.csv', index=False)
    print("Saved: ./experiments/combined_results.csv")